# Let's solve a puzzle!

By Nathan George

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import os
os.chdir('C:\\Users\\nbg05\\Local Documents\\Projects\WhiteHeaven')

from ipywidgets import interact
from tqdm import tqdm

In [2]:
# load the dataset
comparison_scores_dataset = np.load('puzzle/data/comparison_scores.npy')

num_pieces = comparison_scores_dataset.shape[0]
side_indices = [(piece, i) for i in range(4) for piece in range(num_pieces)]

In [3]:
# generate a copy we can modify
working_comparison_scores = comparison_scores_dataset.copy()

for piece1, i1 in side_indices:
    for piece2, i2 in side_indices:
        max_score = max(working_comparison_scores[piece1, i1, piece2, i2], working_comparison_scores[piece2, i2, piece1, i1])
        working_comparison_scores[piece1, i1, piece2, i2] = max_score
        working_comparison_scores[piece2, i2, piece1, i1] = max_score

In [16]:

def get_match_subscore(piece1, side1, piece2, side2, comparison_scores, depth=10, side_sorts=None):
    """gets half of the score for the match between two sides
    """
    
    # rotate the sides
    side1 = (side1 + 1) % 4
    side2 = (side2 + 3) % 4
    
    scores1 = comparison_scores[piece1, side1, :, :]
    scores2 = comparison_scores[piece2, side2, :, :]
    
    if side_sorts is None:
        sort_scores1 = np.stack(np.unravel_index(np.argsort(scores1.flatten())[:depth], scores1.shape), axis=1)
        sort_scores2 = np.stack(np.unravel_index(np.argsort(scores2.flatten())[:depth], scores2.shape), axis=1)
    else:
        sort_scores1 = side_sorts[piece1, side1, :depth]
        sort_scores2 = side_sorts[piece2, side2, :depth]
    
    # check for edge pieces
    is_edge1 = scores1[sort_scores1[0, 0], sort_scores1[0, 1]] == np.inf
    is_edge2 = scores2[sort_scores2[0, 0], sort_scores2[0, 1]] == np.inf
    if is_edge1 and is_edge2:
        return 0, 0
    if is_edge1 or is_edge2:
        return np.inf, 0
    
    best_score = np.inf
    for piece11, side11 in sort_scores1:
        # if scores1[piece11, side11] == np.inf:
        #     break
        for piece22, side22 in sort_scores2:
            # if scores2[piece22, side22] == np.inf:
            #     break
            score = scores1[piece11, side11] + scores2[piece22, side22]
            side11 = (side11 + 1) % 4
            side22 = (side22 + 3) % 4
            score += working_comparison_scores[piece11, side11, piece22, side22]
            
            if score < best_score:
                best_score = score
    
    return best_score, 3

def get_match_score(piece1, edge1, piece2, edge2, comparison_scores, depth=10, side_sorts=None):
    """uses the 3x2 rectangle of pieces around the edge to get how well an edge matches with another edge
    """
    
    num_comparisons = 1
    
    score = comparison_scores[piece1, edge1, piece2, edge2]
    subscore, num = get_match_subscore(piece1, edge1, piece2, edge2, comparison_scores, depth, side_sorts)
    score += subscore
    num_comparisons += num
    subscore, num = get_match_subscore(piece2, edge2, piece1, edge1, comparison_scores, depth, side_sorts)
    score += subscore
    num_comparisons += num
    
    return score / num_comparisons

In [5]:
def get_compound_comparison_scores(piece, i, comparison_scores, side_sorts=None, depth=10, end=100):
    """gets the comparison scores for the 3x2 rectangle of pieces around the edge
    """
    
    # sort the simple scores so we don't have to
    if side_sorts is None:
        scores = comparison_scores[piece, i, :, :]
        indices = np.stack(np.unravel_index(np.argsort(scores.flatten()), scores.shape), axis=1)
    else:
        indices = side_sorts[piece, i]

    compound_scores = np.full((num_pieces, 4), np.inf, dtype=np.float32)

    for piece_, i_ in indices[:end]:
        compound_scores[piece_, i_] = get_match_score(piece, i, piece_, i_, comparison_scores=comparison_scores, depth=depth, side_sorts=side_sorts)
    
    return compound_scores

In [6]:
def compute_confidence(scores):
    scores_flat = scores.flatten()
    sort = np.argsort(scores_flat)
    scores_sorted = scores_flat[sort]
    
    if scores_sorted[3] == np.inf:
        return 0
    
    return (scores_sorted[1] - scores_sorted[0]) / scores_sorted[0]

def get_best_indices(scores):
    scores_flat = scores.flatten()
    sort = np.argsort(scores_flat)
    indices = np.stack(np.unravel_index(sort, scores.shape), axis=1)
    
    return indices[0]

In [72]:
def get_best_matches(compound_comparisons, indices_remaining):
    num_pieces = compound_comparisons.shape[0]
    
    # gets the 2 smallest scores for each piece
    top_matches = np.stack(
        np.unravel_index(
            np.argpartition(compound_comparisons.reshape(num_pieces, 4, -1), kth=1, axis=-1)[:, :, :2], 
            (num_pieces, 4)), 
        axis=-1)

    confidence = np.full((num_pieces, 4), 0, dtype=np.float32)
    for piece, i in indices_remaining:
        best_score = compound_comparisons[piece, i, top_matches[piece, i, 0, 0], top_matches[piece, i, 0, 1]]
        next_score = compound_comparisons[piece, i, top_matches[piece, i, 1, 0], top_matches[piece, i, 1, 1]]
        if next_score == np.inf:
            confidence[piece, i] = 0
            continue
        confidence[piece, i] = (next_score - best_score) / best_score
    
    top_matches = top_matches[:, :, 0, :]
    
    best_matches = []
    best_matches_confidence = []
    
    indices_used = set()
    for piece, i in indices_remaining:
        if (piece, i) in indices_used:
            continue
        
        best_match = (top_matches[piece, i, 0], top_matches[piece, i, 1])
        other_match = (top_matches[best_match][0], top_matches[best_match][1])

        # this piece is the best match for the other piece
        if other_match != (piece, i):
            continue
        
        best_matches.append([piece, i, best_match[0], best_match[1]])
        best_matches_confidence.append(min(confidence[piece, i], confidence[best_match]))
        
        indices_used.add(best_match)
    
    best_matches = np.array(best_matches)
    best_matches_confidence = np.array(best_matches_confidence)
    
    sort = np.argsort(best_matches_confidence)[::-1]
    
    return best_matches[sort], best_matches_confidence[sort]

In [7]:
dumps = {}

In [73]:
solved_pairs = np.zeros((num_pieces, 4, 2), dtype=np.int32)

def solve_side_pairs(original_comparison_scores, num_pieces, sides_per_batch=100):
    # sort scores_sorted
    # compute the compound comparisons for all of the sides
    # take the 100 edges with the highest confidence and add them to the side_pairs
    # modify the working_comparison_scores to remove edges which are not possible
    # resort scores_sorted
    # compute the compound comparisons for all of the remaining sides
    # repeat until all of the sides are filled
    
    working_comparison_scores = original_comparison_scores.copy()
    
    # goal is to fill this with the best matches
    #side_pairs = np.zeros((num_pieces, 4, 2), dtype=np.int32)
    
    indices_remaining = [(piece, i) for piece in range(num_pieces) for i in range(4)]
    
    # remove the edges
    edges = []
    for piece, i in indices_remaining:
        if np.min(working_comparison_scores[piece, i, :, :]) == np.inf:
            edges.append((piece, i))
    
    for edge in edges:
        indices_remaining.remove(edge)
    
    
    while len(indices_remaining) > 0:
    
        # sort
        depth = 10
        
        sort_depth = 50
        side_sorts = np.zeros((num_pieces, 4, sort_depth, 2), dtype=np.int32)
        for piece in range(num_pieces):
            for side in range(4):
                scores = working_comparison_scores[piece, side, :, :]
                side_sorts[piece, side] = np.stack(np.unravel_index(np.argsort(scores.flatten())[:sort_depth], scores.shape), axis=1)

        # compute compound comparisons
        compound_comparisons = np.full((num_pieces, 4, num_pieces, 4), np.inf, dtype=np.float32)
        for piece, i in tqdm(indices_remaining, 'compound comparisons'):
            compound_comparisons[piece, i] = get_compound_comparison_scores(piece, i, comparison_scores=working_comparison_scores, side_sorts=side_sorts, depth=depth, end=sort_depth)
        
        # get the best matches
        matches, confidence = get_best_matches(compound_comparisons, indices_remaining)
        
        # add the 100 best matches to the side_pairs
        i = 0
        while i < sides_per_batch and i < len(matches):
            piece, side, other_piece, other_side = matches[i]
            assert (piece, side) in indices_remaining
            assert (other_piece, other_side) in indices_remaining
            
            solved_pairs[piece, side] = [other_piece, other_side]
            solved_pairs[other_piece, other_side] = [piece, side]
                    
            
            indices_remaining.remove((piece, side))
            indices_remaining.remove((other_piece, other_side))
            
            # modify the working_comparison_scores to remove edges which are not possible
            score = working_comparison_scores[piece, side, other_piece, other_side]
            working_comparison_scores[piece, side, :, :] = np.inf
            working_comparison_scores[:, :, piece, side] = np.inf
            working_comparison_scores[other_piece, other_side, :, :] = np.inf
            working_comparison_scores[:, :, other_piece, other_side] = np.inf
            working_comparison_scores[piece, side, other_piece, other_side] = score
            working_comparison_scores[other_piece, other_side, piece, side] = score
            
            i += 1
    
    return solved_pairs

In [74]:
best_pairs = solve_side_pairs(working_comparison_scores, num_pieces, sides_per_batch=100)

compound comparisons: 100%|██████████| 2/2 [00:00<00:00, 79.10it/s]


In [19]:

#unpack dumps
dumps_location = dumps['location']
dumps_indices_remaining = dumps['indices_remaining']
dumps_working_comparison_scores = dumps['working_comparison_scores']
dumps_side_sorts = dumps['side_sorts']
dumps_compound_comparisons = dumps['compound_comparisons']
dumps_confidence = dumps['confidence']
dumps_best_indices = dumps['best_indices']


In [20]:
get_match_score(47, 0, 153, 0, dumps_working_comparison_scores, side_sorts=dumps_side_sorts, depth=10)

inf

In [21]:
comparisons = get_compound_comparison_scores(47, 0, dumps_working_comparison_scores, side_sorts=None, depth=10, end=100)

In [79]:
# use the solved pairs to generate the puzzle
# grid of all of the pieces

def solve_right_side(puzzle, solved_pairs, x, y):
    piece, side = puzzle[y, x-1]
    piece, side = solved_pairs[piece, side]
    return [piece, (side + 2) % 4]

def solve_bottom_side(puzzle, solved_pairs, x, y):
    piece, side = puzzle[y-1, x]
    piece, side = solved_pairs[piece, (side + 1) % 4]
    return [piece, (side + 1) % 4]

def solve_puzzle(solved_pairs, init_piece, init_side):
    puzzle = np.zeros((40, 25, 2), dtype=np.int32)
    puzzle_bitmask = np.zeros((40, 25), dtype=bool)
    
    puzzle[0, 0] = [init_piece, init_side]
    puzzle_bitmask[0, 0] = True
    
    points = [(y, x) for x in range(25) for y in range(40)]
    
    for y, x in tqdm(points, 'solving'):
        
        if y == 0 and x == 0:
            continue
        
        if y != 0 and x != 0 and puzzle_bitmask[y-1, x] and puzzle_bitmask[y, x-1]:
            # check both sides
            piece_right = solve_right_side(puzzle, solved_pairs, x, y)
            piece_bottom = solve_bottom_side(puzzle, solved_pairs, x, y)
            if not np.array_equal(piece_right, piece_bottom):
                continue
            
            puzzle[y, x] = piece_right
            puzzle_bitmask[y, x] = True
        
        if y == 0 or puzzle_bitmask[y, x-1]:
            # side is the right side of the piece
            puzzle[y, x] = solve_right_side(puzzle, solved_pairs, x, y)
            puzzle_bitmask[y, x] = True
            continue
        
        if x == 0 or puzzle_bitmask[y-1, x]:
            puzzle[y, x] = solve_bottom_side(puzzle, solved_pairs, x, y)
            puzzle_bitmask[y, x] = True
            continue
    
    return puzzle

In [80]:
puzzle = solve_puzzle(best_pairs, 0, 3)

solving: 100%|██████████| 1000/1000 [00:00<00:00, 58988.30it/s]


In [85]:
# check for duplicates in puzzle
puzzle_set = set()
for y in range(40):
    for x in range(25):
        piece, side = puzzle[y, x]
        if piece == 0 and side == 0:
            continue
        if piece in puzzle_set:
            print('duplicate')
        puzzle_set.add(piece)

In [86]:
# save the puzzle
np.save('puzzle/data/puzzle_solved.npy', puzzle)

In [87]:
puzzle_pieces = puzzle[:, :, 0]
puzzle_pieces = puzzle_pieces.T

In [90]:
piece_right = solve_right_side(puzzle, solved_pairs, 20, 38)
piece_bottom = solve_bottom_side(puzzle, solved_pairs, 20, 38)

print(piece_right, piece_bottom)

[275, 1] [901, 2]


In [91]:
# fix missing pieces
puzzle_copy = puzzle.copy()
puzzle_copy[26, 24] = [15, 3]
puzzle_copy[38, 20] = [901, 2]

In [92]:
np.save('puzzle/data/puzzle_solved.npy', puzzle_copy)